# Data cleaning

Our first step will be to clean the data obtained from the ADH PUF flat file. You can start by downloading the flat file avalaible at [this](https://www.census.gov/programs-surveys/ahs/data/2017/ahs-2017-public-use-file--puf-/ahs-2017-national-public-use-file--puf-.html) address and putting the unzipped csv under `./raw_data/` folder at the root of the repository.

## Steps covered in this notebook

1. Selecting variables of interests.
2. Outputting subset csv file.
3. Validating the quality of the data:
 * Weird values
 * Badly formatted data
 * Missing data
 
## Selecting columns of interest
 
We start by importing pandas which we will use to read the orignal csv and select a few columns.

In [363]:
import pandas as pd
ahs_df = pd.read_csv('../raw_data/ahs2017n.csv');

We verify that the data was loaded properly by outputting a couple of rows and the dimensions of the dataframe.

In [364]:
ahs_df.head(5)

,CONTROL,TOTROOMS,PERPOVLVL,COMTYPE,COMCOST,JACPRIMARY,JACSECNDRY,JADEQUACY,JBATHEXCLU,JBATHROOMS,...,JOBDIY28,JOBFUNDS26,JOBFUNDS27,JOBFUNDS28,JOBTYPE26,JOBTYPE27,JOBTYPE28,JOBWORKYR26,JOBWORKYR27,JOBWORKYR28
0,'11000001',8,501,'-6',-6,'0','0','2','0','0',...,'-9','-9','-9','-9','-9','-9','-9','-9','-9','-9'
1,'11000002',7,501,'-6',-6,'0','0','2','0','0',...,'-9','-9','-9','-9','-9','-9','-9','-9','-9','-9'
2,'11000005',8,501,'-6',-6,'0','0','2','0','0',...,'-9','-9','-9','-9','-9','-9','-9','-9','-9','-9'
3,'11000006',5,361,'-6',-6,'0','0','2','0','0',...,'-9','-9','-9','-9','-9','-9','-9','-9','-9','-9'
4,'11000007',8,501,'1',5564,'0','0','2','0','0',...,'-9','-9','-9','-9','-9','-9','-9','-9','-9','-9'


In [365]:
print('The AHS table has {0} rows {1} columns'.format(*ahs_df.shape))

The AHS table has 66752 rows 3180 columns


Before starting this assignment, it was my understanding that the price of the housing units were made available in the dataset in the MARKETVAL column. However the variable of interest, MARKETVAL, which represents the market value of the unit, is given as a boolean <10,000,000\\$ or >=10,000,000\\$. Due to this, I've decided instead to reorient my analysis to answer the following question:

> Given a list of acceptable variable purchase criterions (square footage between X and Y, number of bathrooms between X and Y etc...) find combinations of purchase criterions which seems to "significantly" reduce the incidence of undersirable properties (presence of Cockroaches, rodents, thrash etc...)

After examining the variables available as part of the AHS's minicodebook, I found the following interesting variables:

| Field code | Meaning |
| ---------- | ------- |
| DIVISION   | 1: New England 2: Middle Atlantic 3: East North Central  4: West North Central  5: South Atlantic  6: East South Central  7: West South Central  8: Mountain  9: Pacific |
| BATHROOMS | 01: One full bathroom  02: 1.5 bathrooms  03: 2 bathrooms |
| PLUGS | Every room have an electrical outlet or wall plug that works: 1: YES 2: NO |
| DWNPAYPCT | 0: No down payment 1: Greater than zero to 2 percent 2: 3 to 5 percent 3: 6 to 10 percent 4: 11 to 15 percent 5: 16 to 20 percent 6: 21 to 40 percent 7: 41 to 99 percent 8: 100 percent M or -9: Not reported N or -6: Not applicable |
| YRBUILT | Year in which the house was build encoded in 10 year increments (2010: 2010 to 2017, 2000: 2000 to 2010 etc.)|
| GARAGE | Unit has a garage? 1: Yes 2: No Other values count as not reported|
| PORCH | Unit has porch or deck or balcony or patio? 1: Yes 2: No Other values count as not reported|
| RODENT | Number of times there was signs of rodents in the last 12 months 1: Seen daily in the last 12 months 2: Seen weekly in the last 12 months 3: Seen monthly in the last 12 months 4: Seen a few times in the last 12 months 5: No signs in the last 12 months N or -6: Not applicable |
| ROACH | Same as RODENT for cockroaches (dead or alive)|
| NEARBARCL | Number of buildings with bars on windows within 1/2 block of this housing unit  1: Not more than one building with metal bars 2: More than one building with metal bars 3: No buildings with metal bars M or -9: Not reported N or -6: Not applicable|
| NEARABAND | Number of buildings that are abandoned or vandalized within 1/2 block of this housing unit 1: Not more than one vandalized or abandoned building 2: More than one vandalized or abandoned building 3: There are no vandalized or abandoned buildings 4: No buildings nearby M or -9: Not reported N or -6: Not applicable|
| NEARTRASH |Frequency of trash, litter, or junk in streets, lots, or properties within 1/2 block of this housing unit 1: Small amount of trash2: Large amount of trash3: No trashM or -9: Not reportedN or -6: Not applicable |
| RATINGNH | Rating of neighborhood as a place to live 1 : 10: 1 to 10 M or -9: Not reported N or -6: Not applicable |
| NHQPUBTRN | Agree or Disagree: This neighborhood has good bus, subway, or commuter train service 1: Agree 2: Disagree M or -9: Not reported N or -6: Not applicable |
| NHQRISK | Agree or Disagree: This neighborhood is at high risk for floods or other disasters 1: Agree 2: Disagree M or -9: Not reported N or -6: Not applicable |

As this analysis focuses on housing units (Ideally in the Seattle region) I will first filter the dataset down to the 9th DIVISION as it is the most granular location information available in this public version of the dataset. I will also filter the dataset to houses between 01, 02, 03 options for BATHROOMS.

In [366]:
ahs_pacific_df = ahs_df[ahs_df['DIVISION'] == "'9'"]

In [367]:
print('Our dataset now consists of {0} rows and {1} columns'.format(*ahs_pacific_df.shape)) 

Our dataset now consists of 13330 rows and 3180 columns


In [368]:
ahs_pacific_df = ahs_pacific_df[(ahs_pacific_df['BATHROOMS'] == "'01'") | (ahs_pacific_df['BATHROOMS'] == "'02'") | (ahs_pacific_df['BATHROOMS'] == "'03'")]

In [369]:
print('Our dataset now consists of {0} rows and {1} columns'.format(*ahs_pacific_df.shape)) 

Our dataset now consists of 9916 rows and 3180 columns


We now keep only the variables of interests

In [370]:
features = [
    "BATHROOMS",
    "PLUGS",
    "DWNPAYPCT",
    "YRBUILT",
    "GARAGE",
    "PORCH",
    "RODENT",
    "ROACH",
    "NEARBARCL",
    "NEARABAND",
    "NEARTRASH",
    "RATINGNH",
    "NHQPUBTRN",
    "NHQRISK"
]

housing_df = ahs_pacific_df[features]
print('Our dataset now consists of {0} rows and {1} columns'.format(*housing_df.shape))

Our dataset now consists of 9916 rows and 14 columns


In [371]:
from IPython.display import display,HTML

# The following function was slightly modified from the one found at:
# https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(dfs:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    for df in dfs:
        output += df.style.set_table_attributes("style='display:inline'")._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

def print_tables(df):
    tables = []
    for feature in features:
        my_tab = pd.crosstab(index=df[feature], columns="Count")
        my_tab.columns = ["Count"]
        tables.append(my_tab)      
    display_side_by_side(tables)

In [372]:
print_tables(housing_df)

,Count
BATHROOMS,
'01',4696
'02',1207
'03',4013
,Count
PLUGS,
'-6',15
'1',9712
'2',189
,Count


From the above tables we find that:

| Table | Observation |
| --------- | --------- |
| DIVISIONS | We had check prior to filtering, but none of the entries were missing data |
| BATHROOMS | We had check prior to filtering, but none of the entries were missing data |
| PLUGS | For 15 rows this criteria did not apply, we will exclude those in the following steps. |
| DWNPAYPCT | For 6417 rows this criteria did not apply, additionally, 891 entries did not report this value. We will exclude both of these in the following steps. |
| YRBUILT | All rows have this column populated |
| GARAGE | 41 rows did not report this value. We will exclude them in the following steps. |
| RODENT | For 1087 rows this criteria did not apply, we will exclude those in the following steps. |
| ROACH | For 1087 rows this criteria did not apply, we will exclude those in the following steps. |
| NEARBARCL | For 1087 rows this criteria did not apply, additionally, 470 entries did not report this value. We will exclude both of these in the following steps. |
| NEARABAND | For 1087 rows this criteria did not apply, additionally, 368 entries did not report this value. We will exclude both of these in the following steps. |
| NEARTRASH | For 1087 rows this criteria did not apply, additionally, 310 entries did not report this value. We will exclude both of these in the following steps. |
| RATINGNH | For 1090 rows this criteria did not apply, additionally, 418 entries did not report this value. We will exclude both of these in the following steps. |
| NHQPUBTRN | For 1087 rows this criteria did not apply, additionally, 711 entries did not report this value. We will exclude both of these in the following steps. |
| NHQRISK | For 1087 rows this criteria did not apply, additionally, 423 entries did not report this value. We will exclude both of these in the following steps. |

We will now apply filtering to our dataset. We use a first lambda function below to first convert values to integers. We use a second lambda function to create a filter column for any rows which contains a negative value.

In [373]:
def row_to_integer(row):
    temp = row.copy()
    for index in range(len(row)):
        val = temp.iloc[index]
        if isinstance(val, str):
            temp[index] = int(val.replace('0', '').replace("'", ''))     
    return temp

In [374]:
filtered_housing_df = housing_df.copy()
filtered_housing_numeric_df = filtered_housing_df.apply(lambda row: row_to_integer(row), axis=1)
filtered_housing_numeric_df.head()

,BATHROOMS,PLUGS,DWNPAYPCT,YRBUILT,GARAGE,PORCH,RODENT,ROACH,NEARBARCL,NEARABAND,NEARTRASH,RATINGNH,NHQPUBTRN,NHQRISK
79,2,1,-6,1970,1,1,5,5,3,3,3,6,1,2
81,1,1,-6,2010,1,1,5,5,3,3,3,7,1,2
84,2,2,-6,1980,1,1,5,5,3,3,3,8,1,2
94,3,1,-6,1990,1,1,5,5,3,3,3,8,1,2
97,1,1,-6,1920,2,2,5,5,3,3,3,5,1,2


In [375]:
def negative_filter(row):
    return not any(row < 0)

In [376]:
complete_row_df = filtered_housing_numeric_df[filtered_housing_numeric_df.apply(lambda row: negative_filter(row), axis=1)]
print('Our dataset now consists of {0} rows and {1} columns'.format(*complete_row_df.shape))

Our dataset now consists of 2448 rows and 14 columns


At this point we have considerably reduced our dataset, but the numerical value are not very useful or readable, let's replace them with a string more evocative of their description in the table above.

In [377]:
print_tables(complete_row_df)

,Count
BATHROOMS,
1,528
2,374
3,1546
,Count
PLUGS,
1,2414
2,34
,Count
DWNPAYPCT,


In [379]:
BATHROOMS_labels = {
    1: 1,
    2: 1.5,
    3: 2
}

PLUGS_labels = {
    1: True,
    2: False
}

# We use upper bound of bin
DWNPAYPCT_labels = {
    0: 0,
    1: 2,
    2: 5,
    3: 10,
    4: 15,
    5: 20,
    6: 40,
    7: 99,
    8: 100
}

GARAGE_labels = {
    1: True,
    2: False
}

PORCH_labels = {
    1: True,
    2: False
}

RODENT_labels = {
    1: 'daily',
    2: 'weekly',
    3: 'monthly',
    4: 'few times',
    5: 'no signs'
}

ROACH_labels = {
    1: 'daily',
    2: 'weekly',
    3: 'monthly',
    4: 'few times',
    5: 'no signs'
}

NEARBARCL_labels = {
    1: 'one or less',
    2: 'more than one',
    3: 'none'
}

NEARABAND_labels = {
    1: 'one or less',
    2: 'more than one',
    3: 'none',
    4: 'no building nearby'
}

NEARTRASH_labels = {
    1: 'small',
    2: 'large',
    3: 'none'
}

NHQPUBTRN_labels = {
    1: True,
    2: False
}

NHQRISK_labels = {
    1: True,
    2: False
}

In [380]:
def change_labels(row):
    temp = row.copy()
    temp = temp.astype('float64')
    for col_name in row.index:
        label_map = globals().get('{0}_labels'.format(col_name))
        if label_map != None:
            val = row[col_name]
            temp[col_name] = label_map.get(val)
    return temp

In [381]:
good_labels_df = complete_row_df.apply(lambda row: change_labels(row), axis=1)
good_labels_df.head()

,BATHROOMS,PLUGS,DWNPAYPCT,YRBUILT,GARAGE,PORCH,RODENT,ROACH,NEARBARCL,NEARABAND,NEARTRASH,RATINGNH,NHQPUBTRN,NHQRISK
185,2.0,True,99,1970.0,True,True,no signs,no signs,none,one or less,none,10.0,False,False
398,1.0,True,20,1960.0,False,True,no signs,no signs,none,none,large,10.0,True,False
556,2.0,True,10,1970.0,False,False,few times,weekly,none,none,none,8.0,False,True
568,1.5,True,20,1970.0,False,False,no signs,weekly,none,none,none,5.0,False,False
577,2.0,True,20,2000.0,True,True,no signs,daily,none,none,none,10.0,False,False


While changing the labels the type of some columns where erroneously changed. We use astype to change them back.

In [382]:
good_labels_df = good_labels_df.astype({"YRBUILT": 'int64', "RATINGNH": 'int64'})
good_labels_df.dtypes

BATHROOMS    float64
PLUGS           bool
DWNPAYPCT      int64
YRBUILT        int64
GARAGE          bool
PORCH           bool
RODENT        object
ROACH         object
NEARBARCL     object
NEARABAND     object
NEARTRASH     object
RATINGNH       int64
NHQPUBTRN       bool
NHQRISK         bool
dtype: object

Finally we will rename the columns to make it easier to understand.

In [383]:
good_labels_df.columns = ['nb_bathrooms',
                          'all_rooms_have_plug',
                          'down_payment_percent',
                          'year_built',
                          'has_garage',
                          'has_poarch',
                          'roach_seen',
                          'rodent_seen',
                          'near_bar_on_windows',
                          'near_abandoned_or_vandalized',
                          'near_trash',
                          'neighborhood_rating',
                          'neighborhood_has_good_transit',
                          'neighborhood_is_at_risk_of_natural_disasters'
                         ]

In [384]:
good_labels_df.head()

,nb_bathrooms,all_rooms_have_plug,down_payment_percent,year_built,has_garage,has_poarch,roach_seen,rodent_seen,near_bar_on_windows,near_abandoned_or_vandalized,near_trash,neighborhood_rating,neighborhood_has_good_transit,neighborhood_is_at_risk_of_natural_disasters
185,2.0,True,99,1970,True,True,no signs,no signs,none,one or less,none,10,False,False
398,1.0,True,20,1960,False,True,no signs,no signs,none,none,large,10,True,False
556,2.0,True,10,1970,False,False,few times,weekly,none,none,none,8,False,True
568,1.5,True,20,1970,False,False,no signs,weekly,none,none,none,5,False,False
577,2.0,True,20,2000,True,True,no signs,daily,none,none,none,10,False,False


We can now write the result to a clean data frame csv file.

In [385]:
good_labels_df.to_csv("../clean_data/clean_ahs.csv", encoding='utf-8', index=False)